In [ ]:
#Embedding queries and captions:

#Process captions/queries by lowercasing the text, removing punctuation, and tokenizing words based on white space. Refer to the “bag of words” exercise notebook for efficient code for striping punctuation out of a string

#Take our vocabulary to be all words across all captions in the COCO dataset. Treating each caption as its own “document” compute the inverse document frequency for each word in the vocabulary. Efficiency is important here!

#Make a function that can embed any caption / query text (using GloVe-200 embeddings weighted by IDFs of words across captions)

#An individual word not in the GloVe or IDF vocabulary should yield an embedding vector of just zeros.

In [15]:
from cogworks_data.language import get_data_path
import numpy as np
from pathlib import Path
import json
from collections import Counter

# load COCO metadata
filename = get_data_path("captions_train2014.json")
with Path(filename).open() as f:
    coco_data = json.load(f)

In [10]:
import re, string
punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
from gensim.models import KeyedVectors
filename = "glove.6B.200d.txt.w2v"

# this takes a while to load -- keep this in mind when designing your capstone project
glove = KeyedVectors.load_word2vec_format(get_data_path(filename), binary=False)
    


In [63]:
def preprocess_captions(captions):
    words = []
    for caption in captions:
        cleaned_caption = punc_regex.sub('', caption).lower()
        words = cleaned_caption.split()
        words.extend(words)

    word_counts = Counter(words)
    num_docs = len(captions)
    weights = {word: np.log10(num_docs / count) for word, count in word_counts.items()}
    return weights
idf_weights

{'a': -0.2182920203052396,
 'very': 1.9464092690376869,
 'clean': 2.6571240259330677,
 'and': 0.622669727247448,
 'well': 2.8325015716286086,
 'decorated': 2.5265081564330774,
 'empty': 2.2852868198252354,
 'bathroom': 1.6124536310136068,
 'panoramic': 4.219178855589447,
 'view': 2.0414692495062647,
 'of': 0.4625153671185377,
 'kitchen': 1.6457502274700606,
 'all': 2.34971244550858,
 'its': 1.8540654239618692,
 'appliances': 2.7738860861634747,
 'blue': 1.6361615480318639,
 'white': 1.2126614712147352,
 'with': 0.5849071934286663,
 'butterfly': 3.9639063504861403,
 'themed': 3.7033050118777675,
 'wall': 1.9349737878876523,
 'tiles': 3.2610930070683612,
 'photo': 2.0211819579923103,
 'dining': 2.5938664046277724,
 'room': 1.5052852861126373,
 'graffitied': 4.537937618213859,
 'stop': 1.969638091087808,
 'sign': 1.6298549101392485,
 'across': 2.2152016137439094,
 'the': 0.477305871179939,
 'street': 1.2910084032906541,
 'from': 1.8020062060716715,
 'red': 1.5352676927440299,
 'car': 1.92

In [ ]:
all_captions = [caption_info["caption"] for caption_info in coco_data["annotations"]]
idf_weights = preprocess_captions(all_captions)

In [72]:
def embed_caption(caption, dim=200):
    
    cleanCap = punc_regex.sub('', caption).lower()
    tokens = cleanCap.split()

    real_embeddings = []
    for word in tokens:
        if word in glove:
            idf = idf_weights.get(word)
            real_embeddings.append(idf * glove[word])  # Directly calculate weighted embedding
        else:
            real_embeddings.append(np.zeros(dim))  # Zero vector for missing words
    if not real_embeddings:  #No words
        return np.zeros(dim)
    
    # Sum
    caption_embedding = np.sum(real_embeddings, axis=0) 
    normalized_embedding = caption_embedding / np.linalg.norm(caption_embedding)  # Normalize
    return normalized_embedding

all_embeddings = [embed_caption(caption) for caption in all_captions]

In [73]:
#Make a function that can embed any caption / query text (using GloVe-200 embeddings weighted by IDFs of words across captions)

    
all_embeddings

[array([ 7.06813335e-02, -2.17990186e-02, -8.58557038e-03, -3.49662676e-02,
        -3.97261046e-02, -2.91956519e-03, -4.72545028e-02,  3.51705551e-02,
         2.22443063e-02, -2.09614113e-02,  1.00367926e-02,  3.91419716e-02,
         1.14209484e-02,  2.94075590e-02,  6.53470606e-02,  3.90146151e-02,
        -4.73046042e-02,  4.37835790e-02,  7.06089810e-02, -5.31962514e-02,
         2.86645871e-02,  4.91041452e-01,  8.56937990e-02, -5.72676174e-02,
         6.64593875e-02, -1.47877187e-01, -1.00615565e-02, -7.83390701e-02,
        -7.61269853e-02, -5.70725836e-02,  2.23388858e-02, -3.07240169e-02,
         6.25500381e-02,  3.72684747e-02, -3.93715762e-02, -1.45679070e-02,
        -4.89045307e-02, -7.54101723e-02,  7.93219879e-02,  1.25669017e-02,
         1.28099909e-02,  6.83523566e-02, -5.27764205e-03,  6.55943975e-02,
        -2.08236147e-02, -1.96528547e-02,  1.46776155e-01, -4.82031479e-02,
        -4.62749653e-04,  7.15746433e-02,  1.84787847e-02,  6.49209321e-02,
        -2.1